<a href="https://colab.research.google.com/github/sumuzline/learning/blob/master/sakura_14b_qwen2_5_v1_0_q6k_gguf_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title # Keep this widget playing to prevent Colab from disconnecting you { display-mode: "form" }
#@markdown Press play on the audio player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [2]:
# Mount Google Drive
Mount_GDrive = True # @param {type:"boolean"}
if Mount_GDrive:
  from google.colab import drive
  drive.mount('/content/gdrive')
  ROOT_PATH = "/content/gdrive/MyDrive"
else:
  ROOT_PATH = "/content"
!nvidia-smi

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Wed Dec  4 14:18:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                          

In [3]:
# Install dependencies
!pip install --upgrade pip
!pip install transformers tokenizers
!pip install vllm huggingface-hub flask pyngrok triton torch

In [ ]:
# Run API backend (vLLM)
# Use ngrok for API mapping
from huggingface_hub import hf_hub_download

# Set the ngrok authentication token
ngrokToken = "2cgGHbTHhYWPiFHebWOBBTGYNgS_2kMbG2J43CmKqshPqoYje"

if ngrokToken:
    from pyngrok import conf, ngrok

    # Configure the ngrok authentication token
    conf.get_default().auth_token = ngrokToken
    conf.get_default().monitor_thread = False

    # Start ngrok tunnel with the custom domain
    try:
        ssh_tunnel = ngrok.connect(8001, bind_tls=True, hostname="devoted-hen-awaited.ngrok-free.app")
        public_url = ssh_tunnel.public_url
        print('Custom Domain Address: ' + public_url)
    except Exception as e:
        print(f"Error starting ngrok tunnel: {e}")
else:
    # Download cloudflared tunnel, use cloudflare to set up temporary tunnel (Not Static URL)
    %cd $ROOT_PATH

    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
    !chmod a+x cloudflared
    !./cloudflared tunnel --url localhost:8001




%cd $ROOT_PATH

# Download the model for the first time
!HF_ENDPOINT=https://huggingface.co huggingface-cli download mradermacher/LN-Korean-14B-v0.1-GGUF --local-dir models --include LN-Korean-14B-v0.1.Q6_K.gguf

!RAY_memory_monitor_refresh_ms="0" HF_ENDPOINT=https://huggingface.co OMP_NUM_THREADS=36 VLLM_ATTENTION_BACKEND=XFORMERS vllm serve ./models/LN-Korean-14B-v0.1.Q6_K.gguf --tokenizer Qwen/Qwen2.5-14B-Instruct --dtype float16 --api-key token-abc123 --kv-cache-dtype auto --max-model-len 4096 --tensor-parallel-size 1 --gpu-memory-utilization 0.99 --disable-custom-all-reduce --enforce-eager --use-v2-block-manager --disable-log-requests --host 0.0.0.0 --port 8001 --served-model-name "Qwen2.5-14B-Instruct" &



Error starting ngrok tunnel: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: This domain is reserved for another account.\nFailed to bind the domain 'devoted-hen-awaited.ngrok-free.app' for the account 'sumuzline'.\n\nIf you have reserved this name, make sure that you are using an\nauthtoken credential for the appropriate account.\n\nReserve a name on your dashboard: https://dashboard.ngrok.com/domains/new\r\n\r\nERR_NGROK_320\r\n"}}

/content/gdrive/MyDrive
Fetching 1 files:   0% 0/1 [00:00<?, ?it/s]Downloading 'LN-Korean-14B-v0.1.Q6_K.gguf' to 'models/.cache/huggingface/download/LN-Korean-14B-v0.1.Q6_K.gguf.923c8f16fc53299c6e818da4df2207e3df20a25fb7b93afa37052977a7df0b4c.incomplete'
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:653: UserWarning: Not enough free disk space to download the file. The expected file size is: 12310.16 MB. The target location mod